In [ ]:
import pandas as pd
import numpy as np

In [ ]:
path = "Synthetic_data/experiment_results.csv"
df = pd.read_csv(path)

In [ ]:
df.columns

In [ ]:
iteration_columns = df.filter(regex='^Iteration').copy()
# Convertir a numérico
iteration_columns = iteration_columns.apply(pd.to_numeric, errors='coerce')
# Convertir todos los ceros en NaN
iteration_columns = iteration_columns.replace(0, np.nan)

In [ ]:
import plotly.express as px
# Reduce the number of batteries to a smaller subset for visualization (e.g., first 50 batteries)
subset_data = iteration_columns.head(2000)

# Create a DataFrame to store the iteration values in a long format for the subset
long_data = pd.DataFrame()

for index, row in subset_data.iterrows():
    valid_values = row[row != np.nan]  # Remove padding values (0)
    temp_df = pd.DataFrame({
        'Battery_ID': index,
        'Iteration': np.arange(len(valid_values)),
        'SoH': valid_values.values
    })
    long_data = pd.concat([long_data, temp_df], ignore_index=True)

# Plotting with Plotly Express for the subset
fig = px.line(long_data, x='Iteration', y='SoH', color='Battery_ID', title='State of Health (SoH) Values for 50 Batteries')
fig.update_layout(showlegend=False, xaxis=dict(range=[0, 500]), yaxis=dict(range=[3, 4.3]))  # Hide legend and limit y-axis range
fig.show()


In [ ]:
# Calculate SoH
soh = iteration_columns.div(iteration_columns.iloc[:, 0], axis=0)
soh_thresholds = pd.DataFrame()
# Create columns for the iteration in which SoH reaches 0.98, 0.95, 0.9, 0.85, and 0.8
thresholds = [0.98, 0.95, 0.9, 0.85, 0.8]
for threshold in thresholds:
    soh_thresholds[f'{threshold}'] = soh.apply(lambda row: next((i for i, v in enumerate(row) if v <= threshold), np.nan), axis=1)

soh_thresholds.isna().sum()

In [ ]:
soh_thresholds

In [ ]:
soh
# Create a new dataframe with the specified columns and the SoH thresholds
columns_of_interest = ['charge_c_rate_modulation', 'protocol_choice_prob', 'charge_soc_modulation', 'rest_duration_modulation', 'discharge_factor_modulation', 'discharge_soc_modulation']
df_with_thresholds = df[columns_of_interest].copy()

# Add the SoH thresholds to the dataframe
for threshold in thresholds:
    df_with_thresholds[f'{threshold}'] = soh_thresholds[f'{threshold}']

# Create separate dataframes for each threshold and remove NaN values
dfs_by_threshold = {}
for threshold in thresholds:
    df_threshold = df_with_thresholds[['charge_c_rate_modulation', 'protocol_choice_prob', 'charge_soc_modulation', 'rest_duration_modulation', 'discharge_factor_modulation', 'discharge_soc_modulation', f'{threshold}']].dropna()
    dfs_by_threshold[f'{threshold}'] = df_threshold

# Example: Access the dataframe for SoH 0.8
dfs_by_threshold['0.8']

In [ ]:
dfs_by_threshold.keys()

In [ ]:
!pip install xgboost shap

In [ ]:
import xgboost as xgb
import shap
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Initialize a dictionary to store the most important feature, metrics, and SHAP values for each threshold
most_important_features = {}
metrics = {}
shap_values_dict = {}

# Loop through each threshold and train an XGBoost model
for threshold in thresholds:
    # Prepare the data
    df_threshold = dfs_by_threshold[f'{threshold}']
    X = df_threshold[columns_of_interest]
    y = df_threshold[f'{threshold}']
    
    # Train the XGBoost model
    model = xgb.XGBRegressor()
    model.fit(X, y)
    
    # Calculate SHAP values
    explainer = shap.Explainer(model)
    shap_values = explainer(X)
    shap_values_dict[f'{threshold}'] = shap_values
    
    # Determine the most important feature
    shap_importance = np.abs(shap_values.values).mean(axis=0)
    most_important_feature = columns_of_interest[np.argmax(shap_importance)]
    most_important_features[f'{threshold}'] = most_important_feature
    
    # Predict the values
    y_pred = model.predict(X)
    
    # Calculate metrics
    mae = mean_absolute_error(y, y_pred)
    mse = mean_squared_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    metrics[f'{threshold}'] = {'MAE': mae, 'MSE': mse, 'R²': r2}
    
    # Print the most important feature and metrics for each threshold
    print(f"The most important feature for SoH {threshold} is {most_important_feature}")
    print(f"Metrics for SoH {threshold}: MAE = {mae}, MSE = {mse}, R² = {r2}")
    
    # Plot the SHAP values for the current threshold
    shap.summary_plot(shap_values, X, feature_names=columns_of_interest)
    shap.dependence_plot(most_important_feature, shap_values.values, X, feature_names=columns_of_interest)

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values.values, X, feature_names=columns_of_interest)

In [ ]:
!pip install mlflow


In [ ]:
import mlflow
import mlflow.xgboost
import shap
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pickle
import warnings

# Suppress XGBoost model format warning
warnings.filterwarnings(action='ignore', category=UserWarning, module='xgboost')

# Initialize MLflow experiment
mlflow.set_experiment("New SoH Prediction Experiment")

# Start an MLflow run
with mlflow.start_run():
    # Log parameters
    mlflow.log_param("data_path", path)
    mlflow.log_param("thresholds", thresholds)
    
    # Initialize a dictionary to store the most important feature, metrics, and SHAP values for each threshold
    most_important_features = {}
    metrics = {}
    shap_values_dict = {}

    # Loop through each threshold and train an XGBoost model
    for threshold in thresholds:
        # Prepare the data
        df_threshold = dfs_by_threshold[f'{threshold}']
        X = df_threshold[columns_of_interest]
        y = df_threshold[f'{threshold}']
        
        # Train the XGBoost model
        model = xgb.XGBRegressor()
        model.fit(X, y)
        
        # Calculate SHAP values
        explainer = shap.Explainer(model)
        shap_values = explainer(X)
        shap_values_dict[f'{threshold}'] = shap_values
        
        # Determine the most important feature
        shap_importance = np.abs(shap_values.values).mean(axis=0)
        most_important_feature = columns_of_interest[np.argmax(shap_importance)]
        most_important_features[f'{threshold}'] = most_important_feature
        
        # Predict the values
        y_pred = model.predict(X)
        
        # Calculate metrics
        mae = mean_absolute_error(y, y_pred)
        mse = mean_squared_error(y, y_pred)
        r2 = r2_score(y, y_pred)
        metrics[f'{threshold}'] = {'MAE': mae, 'MSE': mse, 'R²': r2}
        
        # Log metrics
        # Log the model with input example
        input_example = X.iloc[:5]
        mlflow.xgboost.log_model(model, f"model_{threshold}", input_example=input_example)
        mlflow.log_metric(f"R2_{threshold}", r2)
        
        # Log the model
        mlflow.xgboost.log_model(model, f"model_{threshold}")
        
        # Print the most important feature and metrics for each threshold
        print(f"The most important feature for SoH {threshold} is {most_important_feature}")
        print(f"Metrics for SoH {threshold}: MAE = {mae}, MSE = {mse}, R² = {r2}")
        
        # Plot the SHAP values for the current threshold
        shap.summary_plot(shap_values, X, feature_names=columns_of_interest)
        shap.dependence_plot(most_important_feature, shap_values.values, X, feature_names=columns_of_interest)
    
    # Log the most important features
    mlflow.log_dict(most_important_features, "most_important_features.json")
    
    # Log the SHAP values
    for threshold, shap_values in shap_values_dict.items():
        shap_values_file = f"shap_values_{threshold}.pkl"
        with open(shap_values_file, "wb") as f:
            pickle.dump(shap_values, f)
        mlflow.log_artifact(shap_values_file)

    # Log the experimental setup
    mlflow.log_param("experimental_setup", {
        "data_path": path,
        "columns_of_interest": columns_of_interest,
        "thresholds": thresholds
    })

    # Log the preprocessing steps
    mlflow.log_param("preprocessing_steps", {
        "iteration_columns_conversion": "Converted to numeric and replaced zeros with NaN",
        "soh_calculation": "Calculated SoH and created thresholds",
        "dataframe_creation": "Created dataframes for each threshold and removed NaN values"
    })

    # Log the model training and evaluation details
    mlflow.log_param("model_training", {
        "model_type": "XGBoost",
        "evaluation_metrics": ["MAE", "MSE", "R²"]
    })

    # Log the most important features
    mlflow.log_dict(most_important_features, "most_important_features.json")

    # Log the SHAP values
    for threshold, shap_values in shap_values_dict.items():
        shap_values_file = f"shap_values_{threshold}.pkl"
        with open(shap_values_file, "wb") as f:
            pickle.dump(shap_values, f)
        mlflow.log_artifact(shap_values_file)

    # Track the specific file of the experimental result
    # This will log the experimental result file to MLflow
    mlflow.log_artifact(path)


In [ ]:
import mlflow
import mlflow.xgboost
import mlflow.data
import shap
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import pickle
import warnings
import matplotlib.pyplot as plt

# Suppress XGBoost model format warning
warnings.filterwarnings(action='ignore', category=UserWarning, module='xgboost')

# Initialize MLflow experiment
mlflow.set_experiment("New SoH Prediction Experiment")

# Start an MLflow run
with mlflow.start_run() as parent_run:
    # Log parameters
    path = "Synthetic_data/experiment_results.csv"
    mlflow.log_param("data_path", path)
    
    # Load data
    df = pd.read_csv(path)
    mlflow.log_param("data_shape", df.shape)
    
    # Preprocessing steps
    iteration_columns = df.filter(regex='^Iteration').copy()
    mlflow.log_param("iteration_columns_shape", iteration_columns.shape)
    
    # Convert to numeric
    iteration_columns = iteration_columns.apply(pd.to_numeric, errors='coerce')
    mlflow.log_param("iteration_columns_conversion", "Converted to numeric")
    
    # Replace zeros with NaN
    iteration_columns = iteration_columns.replace(0, np.nan)
    mlflow.log_param("iteration_columns_replace_zeros", "Replaced zeros with NaN")
    
    # Calculate SoH
    soh = iteration_columns.div(iteration_columns.iloc[:, 0], axis=0)
    mlflow.log_param("soh_calculation", "Calculated SoH")
    
    # Create SoH thresholds
    soh_thresholds = pd.DataFrame()
    thresholds = [0.98, 0.95, 0.9, 0.85, 0.8]
    for threshold in thresholds:
        soh_thresholds[str(threshold)] = soh.apply(
            lambda row: next((i for i, v in enumerate(row) if v <= threshold), len(row)),
            axis=1
        )
    soh_thresholds = soh_thresholds.astype(float)
    mlflow.log_param("soh_thresholds_creation", "Created SoH thresholds")
    
    # Log the number of NaN values in SoH thresholds
    mlflow.log_param("soh_thresholds_nan_count", soh_thresholds.isna().sum().to_dict())
    
    # Add the SoH thresholds to the dataframe
    columns_of_interest = [
        'charge_c_rate_modulation', 'protocol_choice_prob', 'charge_soc_modulation',
        'rest_duration_modulation', 'discharge_factor_modulation', 'discharge_soc_modulation'
    ]
    df_with_thresholds = df[columns_of_interest].copy()
    for threshold in thresholds:
        df_with_thresholds[str(threshold)] = soh_thresholds[str(threshold)]
    mlflow.log_param("df_with_thresholds_shape", df_with_thresholds.shape)
    
    # Create separate dataframes for each threshold and remove NaN values
    dfs_by_threshold = {}
    for threshold in thresholds:
        df_threshold = df_with_thresholds[columns_of_interest + [str(threshold)]].dropna()
        dfs_by_threshold[f'{threshold}'] = df_threshold
    
    def process_threshold(threshold, df_threshold):
        with mlflow.start_run(nested=True) as child_run:
            # Prepare the data
            X = df_threshold[columns_of_interest]
            y = df_threshold[f'{threshold}']
            
            # Split data into train and test sets
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.2, random_state=42
            )
            
            # Log the training Dataset
            train_data = X_train.copy()
            train_data['label'] = y_train
            train_dataset = mlflow.data.from_pandas(train_data, targets='label')
            mlflow.log_input(train_dataset, context='training')
            
            # Train the XGBoost model
            model = xgb.XGBRegressor()
            model.fit(X_train, y_train)
            
            # Log the model
            mlflow.xgboost.log_model(
                model, artifact_path=f"model_{threshold}", input_example=X_test
            )
            
            # Predict the values
            y_pred = model.predict(X_test)
            
            # Build the evaluation dataset
            eval_data = X_test.copy()
            eval_data['label'] = y_test
            eval_data['predictions'] = y_pred  # Include predictions
            
            # Create the PandasDataset for use in mlflow evaluate
            pd_dataset = mlflow.data.from_pandas(
                eval_data.drop(columns=['predictions']), targets='label'
            )
            
            # Log the evaluation Dataset
            mlflow.log_input(pd_dataset, context='evaluation')
            
            # Evaluate the model
            model_uri = f"runs:/{child_run.info.run_id}/model_{threshold}"
            result = mlflow.evaluate(
                model=model_uri,
                data=pd_dataset,
                targets=None,
                model_type='regressor',
                evaluators='default'
            )
            
            # Calculate metrics
            mae = mean_absolute_error(y_test, y_pred)
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            
            # Log metrics
            mlflow.log_metric("MAE", mae)
            mlflow.log_metric("MSE", mse)
            mlflow.log_metric("R2", r2)
            
            # Calculate SHAP values on the test data
            explainer = shap.Explainer(model)
            shap_values = explainer(X_test)
            
            # Determine the most important feature
            shap_importance = np.abs(shap_values.values).mean(axis=0)
            most_important_feature = columns_of_interest[np.argmax(shap_importance)]
            
            # Log the most important feature
            mlflow.log_param("most_important_feature", most_important_feature)
            
            # Print the most important feature and metrics
            print(f"The most important feature for SoH {threshold} is {most_important_feature}")
            print(f"Metrics for SoH {threshold}: MAE = {mae}, MSE = {mse}, R² = {r2}")
            
            # Plot the SHAP values for the current threshold
            shap.summary_plot(shap_values, X_test, feature_names=columns_of_interest, show=False)
            plt.savefig(f"shap_summary_plot_{threshold}.png", bbox_inches='tight')
            mlflow.log_artifact(f"shap_summary_plot_{threshold}.png")
            plt.clf()
            
            shap.dependence_plot(
                most_important_feature, shap_values.values, X_test,
                feature_names=columns_of_interest, show=False
            )
            plt.savefig(f"shap_dependence_plot_{threshold}.png", bbox_inches='tight')
            mlflow.log_artifact(f"shap_dependence_plot_{threshold}.png")
            plt.clf()
            
            # Save the SHAP values
            shap_values_file = f"shap_values_{threshold}.pkl"
            with open(shap_values_file, "wb") as f:
                pickle.dump(shap_values, f)
            mlflow.log_artifact(shap_values_file)

    
    # Loop through each threshold and process
    for threshold in thresholds:
        df_threshold = dfs_by_threshold[f'{threshold}']
        process_threshold(threshold, df_threshold)
    
    # Log the experimental setup
    mlflow.log_param("experimental_setup", {
        "data_path": path,
        "columns_of_interest": columns_of_interest,
        "thresholds": thresholds
    })
    
    # Log the dataset used
    mlflow.log_param("dataset_used", path)
    
    # Track the specific file of the experimental result
    mlflow.log_artifact(path)

In [ ]:
!pip install -U "ray[data,train,tune,serve]"

In [ ]:
import mlflow
import mlflow.xgboost
import shap
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pickle
import warnings
from ray import tune, train
from ray.tune.schedulers import ASHAScheduler

# Suppress XGBoost model format warning
warnings.filterwarnings(action='ignore', category=UserWarning, module='xgboost')

# Initialize MLflow experiment
mlflow.set_experiment("New SoH Prediction Experiment")

# Define the search space for hyperparameters
search_space = {
    "learning_rate": tune.loguniform(0.01, 0.1),
    "max_depth": tune.randint(3, 10),
    "min_child_weight": tune.randint(1, 6),
    "subsample": tune.uniform(0.5, 1.0),
    "colsample_bytree": tune.uniform(0.5, 1.0)
}

# Define the training function
def train_model(config, threshold, X, y):
    import xgboost as xgb
    from sklearn.metrics import mean_absolute_error
    # No need to import tune again if already imported globally
    model = xgb.XGBRegressor(
        learning_rate=config["learning_rate"],
        max_depth=config["max_depth"],
        min_child_weight=config["min_child_weight"],
        subsample=config["subsample"],
        colsample_bytree=config["colsample_bytree"]
    )
    model.fit(X, y)
    y_pred = model.predict(X)
    mae = mean_absolute_error(y, y_pred)
    train.report({"mae":mae})

# Define or load necessary variables before running the script
# For example:
# path = "path_to_your_data.csv"
# thresholds = [0.8, 0.85, 0.9]
# columns_of_interest = ["feature1", "feature2", "feature3"]
# dfs_by_threshold = {"0.8": df1, "0.85": df2, "0.9": df3}

# Start an MLflow run
with mlflow.start_run():
    # Log parameters
    mlflow.log_param("data_path", path)
    mlflow.log_param("thresholds", thresholds)
    
    # Initialize dictionaries to store results
    most_important_features = {}
    metrics = {}
    shap_values_dict = {}

    # Loop through each threshold and train an XGBoost model
    for threshold in thresholds:
        # Prepare the data
        df_threshold = dfs_by_threshold[str(threshold)]
        X = df_threshold[columns_of_interest]
        y = df_threshold[str(threshold)]
        
        # Perform hyperparameter tuning with Ray Tune
        scheduler = ASHAScheduler(
            metric="mae",
            mode="min",
            max_t=10,
            grace_period=1,
            reduction_factor=2
        )
        
        # Sample a subset of the data to reduce memory usage
        X_sample = X.sample(frac=0.5, random_state=42)
        y_sample = y.loc[X_sample.index]

        analysis = tune.run(
            tune.with_parameters(train_model, threshold=threshold, X=X_sample, y=y_sample),
            config=search_space,
            num_samples=10,
            scheduler=scheduler,
             max_concurrent_trials=1  # Limit the number of concurrent trials
            # For newer versions of Ray, limit concurrency using ConcurrencyLimiter if needed
        )
        
        # Get the best hyperparameters
        best_config = analysis.get_best_config(metric="mae", mode="min")
        
        # Train the XGBoost model with the best hyperparameters
        model = xgb.XGBRegressor(
            learning_rate=best_config["learning_rate"],
            max_depth=best_config["max_depth"],
            min_child_weight=best_config["min_child_weight"],
            subsample=best_config["subsample"],
            colsample_bytree=best_config["colsample_bytree"]
        )
        model.fit(X, y)
        
        # Calculate SHAP values
        explainer = shap.Explainer(model)
        shap_values = explainer(X)
        shap_values_dict[str(threshold)] = shap_values
        
        # Determine the most important feature
        shap_importance = np.abs(shap_values.values).mean(axis=0)
        most_important_feature = columns_of_interest[np.argmax(shap_importance)]
        most_important_features[str(threshold)] = most_important_feature
        
        # Predict the values
        y_pred = model.predict(X)
        
        # Calculate metrics
        mae = mean_absolute_error(y, y_pred)
        mse = mean_squared_error(y, y_pred)
        r2 = r2_score(y, y_pred)
        metrics[str(threshold)] = {'MAE': mae, 'MSE': mse, 'R²': r2}
        
        # Log metrics
        # Log the model with input example
        input_example = X.iloc[:5]
        mlflow.xgboost.log_model(model, f"model_{threshold}", input_example=input_example)
        mlflow.log_metric(f"R2_{threshold}", r2)
        
        # Print the most important feature and metrics for each threshold
        print(f"The most important feature for SoH {threshold} is {most_important_feature}")
        print(f"Metrics for SoH {threshold}: MAE = {mae}, MSE = {mse}, R² = {r2}")
        
        # Plot the SHAP values for the current threshold
        # Save the plots instead of displaying them
        shap.summary_plot(shap_values, X, feature_names=columns_of_interest, show=False)
        plt.savefig(f"shap_summary_{threshold}.png")
        mlflow.log_artifact(f"shap_summary_{threshold}.png")
        plt.close()

        shap.dependence_plot(most_important_feature, shap_values.values, X, feature_names=columns_of_interest, show=False)
        plt.savefig(f"shap_dependence_{threshold}.png")
        mlflow.log_artifact(f"shap_dependence_{threshold}.png")
        plt.close()
    
    # Log the most important features
    mlflow.log_dict(most_important_features, "most_important_features.json")
    
    # Log the SHAP values
    for threshold, shap_values in shap_values_dict.items():
        shap_values_file = f"shap_values_{threshold}.pkl"
        with open(shap_values_file, "wb") as f:
            pickle.dump(shap_values, f)
        mlflow.log_artifact(shap_values_file)

    # Log the experimental setup
    mlflow.log_param("experimental_setup", {
        "data_path": path,
        "columns_of_interest": columns_of_interest,
        "thresholds": thresholds
    })

    # Log the preprocessing steps
    mlflow.log_param("preprocessing_steps", {
        "iteration_columns_conversion": "Converted to numeric and replaced zeros with NaN",
        "soh_calculation": "Calculated SoH and created thresholds",
        "dataframe_creation": "Created dataframes for each threshold and removed NaN values"
    })

    # Log the model training and evaluation details
    mlflow.log_param("model_training", {
        "model_type": "XGBoost",
        "evaluation_metrics": ["MAE", "MSE", "R²"]
    })

    # Track the specific file of the experimental result
    # This will log the experimental result file to MLflow
    mlflow.log_artifact(path)


In [ ]:



# Suppress XGBoost model format warning
warnings.filterwarnings(action='ignore', category=UserWarning, module='xgboost')

# Initialize MLflow experiment
mlflow.set_experiment("New SoH Prediction Experiment")

# Define the search space for hyperparameters, including 'threshold'
search_space = {
    "learning_rate": hyperopt.hp.loguniform("learning_rate", np.log(0.01), np.log(0.1)),
    "max_depth": hyperopt.hp.randint("max_depth", 3, 10),
    "min_child_weight": hyperopt.hp.randint("min_child_weight", 1, 6),
    "subsample": hyperopt.hp.uniform("subsample", 0.5, 1.0),
    "colsample_bytree": hyperopt.hp.uniform("colsample_bytree", 0.5, 1.0),
    "threshold": hyperopt.hp.choice("threshold", [0.98, 0.95, 0.9, 0.85, 0.8])  # Threshold as a hyperparameter
}

# Define the training function
def train_model(config, data, columns_of_interest):
    import xgboost as xgb
    from sklearn.metrics import r2_score
    from sklearn.model_selection import train_test_split
    import pandas as pd
    import numpy as np

    # Extract the threshold from the config
    threshold = config["threshold"]
    
    # Preprocess data based on the threshold
    # Calculate SoH (State of Health) based on the iteration columns
    iteration_columns = data.filter(regex='^Iteration').copy()
    iteration_columns = iteration_columns.apply(pd.to_numeric, errors='coerce')
    iteration_columns = iteration_columns.replace(0, np.nan)
    soh = iteration_columns.div(iteration_columns.iloc[:, 0], axis=0)
    
    # Calculate the target variable 'y' based on the threshold
    soh_threshold = soh.apply(
        lambda row: next((i for i, v in enumerate(row) if v <= threshold), len(row)),
        axis=1
    )
    soh_threshold = soh_threshold.astype(float)
    
    # Combine features and target into a single DataFrame
    df_with_threshold = data[columns_of_interest].copy()
    df_with_threshold['target'] = soh_threshold
    df_with_threshold = df_with_threshold.dropna()
    
    # Split features and target
    X = df_with_threshold[columns_of_interest]
    y = df_with_threshold['target']
    
    # Split data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # Initialize the XGBoost regressor with hyperparameters from 'config'
    model = xgb.XGBRegressor(
        learning_rate=config["learning_rate"],
        max_depth=int(config["max_depth"]),
        min_child_weight=int(config["min_child_weight"]),
        subsample=config["subsample"],
        colsample_bytree=config["colsample_bytree"],
        objective='reg:squarederror',
        seed=42,
        n_jobs=1  # Limit to 1 CPU per trial to prevent over-subscription
    )
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Predict on the validation set
    y_pred = model.predict(X_val)
    
    # Calculate R² on the validation set
    r2 = r2_score(y_val, y_pred)
    
    # Report the metric to Ray Tune
    train.report({"r2":r2})

# Start an MLflow run
with mlflow.start_run():
    # Log parameters
    path = "Synthetic_data/experiment_results.csv"
    mlflow.log_param("data_path", path)
    
    # Load data
    df = pd.read_csv(path)
    mlflow.log_param("data_shape", df.shape)
    
    # Define the columns of interest (features)
    columns_of_interest = [
        'charge_c_rate_modulation', 'protocol_choice_prob', 'charge_soc_modulation',
        'rest_duration_modulation', 'discharge_factor_modulation', 'discharge_soc_modulation'
    ]
    
    # Log the features used
    mlflow.log_param("columns_of_interest", columns_of_interest)
    
    # Prepare the data (this will be passed to the training function)
    data = df.copy()
    
    # Initialize HyperOptSearch
    hyperopt_search = HyperOptSearch(
        space=search_space,
        metric="r2",
        mode="max"
    )
    
    # Initialize HyperBandScheduler for iterative, race-like optimization
    scheduler = HyperBandScheduler(
        metric="r2",
        mode="max",
        max_t=10,
        reduction_factor=3
    )
    
    # Run hyperparameter tuning
    analysis = tune.run(
        tune.with_parameters(train_model, data=data, columns_of_interest=columns_of_interest),
        search_alg=hyperopt_search,
        num_samples=200,  # Number of hyperparameter configurations to try
        scheduler=scheduler,
        resources_per_trial={"cpu": 1},  # Adjust based on your resources
        verbose=1
    )
    
    # Get the best hyperparameters
    best_config = analysis.get_best_config(metric="r2", mode="max")
    
    # Log the best hyperparameters
    mlflow.log_params(best_config)
    
    # Extract the best threshold
    best_threshold = best_config["threshold"]
    
    # Recompute the target variable 'y' using the best threshold
    iteration_columns = data.filter(regex='^Iteration').copy()
    iteration_columns = iteration_columns.apply(pd.to_numeric, errors='coerce')
    iteration_columns = iteration_columns.replace(0, np.nan)
    soh = iteration_columns.div(iteration_columns.iloc[:, 0], axis=0)
    
    soh_threshold = soh.apply(
        lambda row: next((i for i, v in enumerate(row) if v <= best_threshold), len(row)),
        axis=1
    )
    soh_threshold = soh_threshold.astype(float)
    
    # Prepare the final dataset with the best threshold
    df_with_threshold = data[columns_of_interest].copy()
    df_with_threshold['target'] = soh_threshold
    df_with_threshold = df_with_threshold.dropna()
    
    # Split features and target
    X = df_with_threshold[columns_of_interest]
    y = df_with_threshold['target']
    
    # Train the final model on the entire dataset with best hyperparameters
    best_model = xgb.XGBRegressor(
        learning_rate=best_config["learning_rate"],
        max_depth=int(best_config["max_depth"]),
        min_child_weight=int(best_config["min_child_weight"]),
        subsample=best_config["subsample"],
        colsample_bytree=best_config["colsample_bytree"],
        objective='reg:squarederror',
        seed=42
    )
    
    best_model.fit(X, y)
    
    # Calculate SHAP values
    explainer = shap.Explainer(best_model)
    shap_values = explainer(X)
    
    # Determine the most important feature
    shap_importance = np.abs(shap_values.values).mean(axis=0)
    most_important_feature = columns_of_interest[np.argmax(shap_importance)]
    
    # Log the most important feature
    mlflow.log_param("most_important_feature", most_important_feature)
    
    # Predict the values
    y_pred = best_model.predict(X)
    
    # Calculate metrics
    mae = mean_absolute_error(y, y_pred)
    mse = mean_squared_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    
    # Log metrics
    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("MSE", mse)
    mlflow.log_metric("R2", r2)
    
    # Log the model with input example
    input_example = X.iloc[:5]
    mlflow.xgboost.log_model(best_model, "best_model", input_example=input_example)
    
    # Print the most important feature and metrics
    print(f"The most important feature is {most_important_feature}")
    print(f"Metrics: MAE = {mae:.4f}, MSE = {mse:.4f}, R² = {r2:.4f}")
    
    # Plot the SHAP values
    shap.summary_plot(shap_values, X, feature_names=columns_of_interest, show=False)
    plt.savefig("shap_summary.png", bbox_inches='tight')
    mlflow.log_artifact("shap_summary.png")
    plt.close()
    
    shap.dependence_plot(
        most_important_feature, shap_values.values, X,
        feature_names=columns_of_interest, show=False
    )
    plt.savefig("shap_dependence.png", bbox_inches='tight')
    mlflow.log_artifact("shap_dependence.png")
    plt.close()
    
    # Save the SHAP values
    shap_values_file = "shap_values.pkl"
    with open(shap_values_file, "wb") as f:
        pickle.dump(shap_values, f)
    mlflow.log_artifact(shap_values_file)
    
    # Log the experimental setup
    mlflow.log_param("experimental_setup", {
        "data_path": path,
        "columns_of_interest": columns_of_interest
    })
    
    # Log the preprocessing steps
    mlflow.log_param("preprocessing_steps", {
        "iteration_columns_conversion": "Converted to numeric and replaced zeros with NaN",
        "soh_calculation": "Calculated SoH and created thresholds"
    })
    
    # Log the model training and evaluation details
    mlflow.log_param("model_training", {
        "model_type": "XGBoost",
        "evaluation_metrics": ["MAE", "MSE", "R²"],
        "hyperparameter_tuning": "Used Ray Tune with HyperOptSearch and HyperBandScheduler, maximizing R²"
    })
    
    # Track the specific file of the experimental result
    mlflow.log_artifact(path)


In [ ]:

import hyperopt
import mlflow
import mlflow.xgboost
import shap
import xgboost as xgb
import numpy as np
import pandas as pd
import pickle
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from ray import tune, train
from ray.tune.schedulers import HyperBandScheduler
from ray.tune.search.hyperopt import HyperOptSearch

# Suppress XGBoost model format warning
warnings.filterwarnings(action='ignore', category=UserWarning, module='xgboost')

# Initialize MLflow experiment
mlflow.set_experiment("New SoH Prediction Experiment")

# Define the search space for hyperparameters, including 'threshold'
search_space = {
    "learning_rate": hyperopt.hp.loguniform("learning_rate", np.log(0.01), np.log(0.1)),
    "max_depth": hyperopt.hp.randint("max_depth", 3, 10),
    "min_child_weight": hyperopt.hp.randint("min_child_weight", 1, 6),
    "subsample": hyperopt.hp.uniform("subsample", 0.5, 1.0),
    "colsample_bytree": hyperopt.hp.uniform("colsample_bytree", 0.5, 1.0),
    "threshold": hyperopt.hp.choice("threshold", [0.98, 0.95, 0.9, 0.85, 0.8])  # Threshold as a hyperparameter
}

# Start an MLflow run
with mlflow.start_run():
    # Log parameters
    path = "Synthetic_data/experiment_results.csv"
    mlflow.log_param("data_path", path)
    
    # Load data
    df = pd.read_csv(path)
    mlflow.log_param("data_shape", df.shape)
    
    # Define the columns of interest (features)
    columns_of_interest = [
        'charge_c_rate_modulation', 'protocol_choice_prob', 'charge_soc_modulation',
        'rest_duration_modulation', 'discharge_factor_modulation', 'discharge_soc_modulation'
    ]
    
    # Log the features used
    mlflow.log_param("columns_of_interest", columns_of_interest)
    
    # Prepare the data
    data = df.copy()
    
    # Split the data into training+validation (90%) and test (10%) sets
    train_val_data, test_data = train_test_split(
        data, test_size=0.1, random_state=42
    )
    
    # Log the sizes of the datasets
    mlflow.log_param("train_val_data_shape", train_val_data.shape)
    mlflow.log_param("test_data_shape", test_data.shape)
    
    # Define the training function
    def train_model(config, train_val_data, columns_of_interest):
        import xgboost as xgb
        from sklearn.metrics import r2_score
        from sklearn.model_selection import train_test_split
        import pandas as pd
        import numpy as np
        
        # Extract the threshold from the config
        threshold = config["threshold"]
        
        # Preprocess data based on the threshold
        # Calculate SoH (State of Health) based on the iteration columns
        iteration_columns = train_val_data.filter(regex='^Iteration').copy()
        iteration_columns = iteration_columns.apply(pd.to_numeric, errors='coerce')
        iteration_columns = iteration_columns.replace(0, np.nan)
        soh = iteration_columns.div(iteration_columns.iloc[:, 0], axis=0)
        
        # Calculate the target variable 'y' based on the threshold
        soh_threshold = soh.apply(
            lambda row: next((i for i, v in enumerate(row) if v <= threshold), len(row)),
            axis=1
        )
        soh_threshold = soh_threshold.astype(float)
        
        # Combine features and target into a single DataFrame
        df_with_threshold = train_val_data[columns_of_interest].copy()
        df_with_threshold['target'] = soh_threshold
        df_with_threshold = df_with_threshold.dropna()
        
        # Split features and target
        X = df_with_threshold[columns_of_interest]
        y = df_with_threshold['target']
        
        # Split data into training and validation sets (within train_val_data)
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
        
        # Initialize the XGBoost regressor with hyperparameters from 'config'
        model = xgb.XGBRegressor(
            learning_rate=config["learning_rate"],
            max_depth=int(config["max_depth"]),
            min_child_weight=int(config["min_child_weight"]),
            subsample=config["subsample"],
            colsample_bytree=config["colsample_bytree"],
            objective='reg:squarederror',
            seed=42,
            n_jobs=1  # Limit to 1 CPU per trial to prevent over-subscription
        )
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Predict on the validation set
        y_pred = model.predict(X_val)
        
        # Calculate R² on the validation set
        r2 = r2_score(y_val, y_pred)
        
        # Report the metric to Ray Tune
        train.report({"r2":r2})
    
    # Initialize HyperOptSearch
    hyperopt_search = HyperOptSearch(
        space=search_space,
        metric="r2",
        mode="max"
    )
    
    # Initialize HyperBandScheduler for iterative, race-like optimization
    scheduler = HyperBandScheduler(
        metric="r2",
        mode="max",
        max_t=10,
        reduction_factor=3
    )
    
    # Run hyperparameter tuning
    analysis = tune.run(
        tune.with_parameters(train_model, train_val_data=train_val_data, columns_of_interest=columns_of_interest),
        search_alg=hyperopt_search,
        num_samples=200,  # Number of hyperparameter configurations to try
        scheduler=scheduler,
        resources_per_trial={"cpu": 1},  # Adjust based on your resources
        verbose=1
    )
    
    # Get the best hyperparameters
    best_config = analysis.get_best_config(metric="r2", mode="max")
    
    # Log the best hyperparameters
    mlflow.log_params(best_config)
    
    # Extract the best threshold
    best_threshold = best_config["threshold"]
    
    # Recompute the target variable 'y' using the best threshold on the combined train_val_data
    iteration_columns = train_val_data.filter(regex='^Iteration').copy()
    iteration_columns = iteration_columns.apply(pd.to_numeric, errors='coerce')
    iteration_columns = iteration_columns.replace(0, np.nan)
    soh = iteration_columns.div(iteration_columns.iloc[:, 0], axis=0)
    
    soh_threshold = soh.apply(
        lambda row: next((i for i, v in enumerate(row) if v <= best_threshold), len(row)),
        axis=1
    )
    soh_threshold = soh_threshold.astype(float)
    
    # Prepare the final dataset with the best threshold
    df_with_threshold = train_val_data[columns_of_interest].copy()
    df_with_threshold['target'] = soh_threshold
    df_with_threshold = df_with_threshold.dropna()
    
    # Split features and target
    X_train_val = df_with_threshold[columns_of_interest]
    y_train_val = df_with_threshold['target']
    
    # Retrain the final model on the combined training and validation data
    best_model = xgb.XGBRegressor(
        learning_rate=best_config["learning_rate"],
        max_depth=int(best_config["max_depth"]),
        min_child_weight=int(best_config["min_child_weight"]),
        subsample=best_config["subsample"],
        colsample_bytree=best_config["colsample_bytree"],
        objective='reg:squarederror',
        seed=42
    )
    
    best_model.fit(X_train_val, y_train_val)
    
    # Prepare the test set for evaluation
    # Preprocess the test data using the best threshold
    iteration_columns_test = test_data.filter(regex='^Iteration').copy()
    iteration_columns_test = iteration_columns_test.apply(pd.to_numeric, errors='coerce')
    iteration_columns_test = iteration_columns_test.replace(0, np.nan)
    soh_test = iteration_columns_test.div(iteration_columns_test.iloc[:, 0], axis=0)
    
    soh_threshold_test = soh_test.apply(
        lambda row: next((i for i, v in enumerate(row) if v <= best_threshold), len(row)),
        axis=1
    )
    soh_threshold_test = soh_threshold_test.astype(float)
    
    # Prepare the test dataset with the best threshold
    df_test_with_threshold = test_data[columns_of_interest].copy()
    df_test_with_threshold['target'] = soh_threshold_test
    df_test_with_threshold = df_test_with_threshold.dropna()
    
    # Split features and target for the test set
    X_test = df_test_with_threshold[columns_of_interest]
    y_test = df_test_with_threshold['target']
    
    # Evaluate the model on the test set
    y_pred_test = best_model.predict(X_test)
    
    # Calculate metrics on the test set
    mae_test = mean_absolute_error(y_test, y_pred_test)
    mse_test = mean_squared_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)
    
    # Log test metrics
    mlflow.log_metric("Test_MAE", mae_test)
    mlflow.log_metric("Test_MSE", mse_test)
    mlflow.log_metric("Test_R2", r2_test)
    
    # Print the test metrics
    print(f"Test Metrics: MAE = {mae_test:.4f}, MSE = {mse_test:.4f}, R² = {r2_test:.4f}")
    
    # Calculate SHAP values on the combined training and validation data
    explainer = shap.Explainer(best_model)
    shap_values = explainer(X_train_val)
    
    # Determine the most important feature
    shap_importance = np.abs(shap_values.values).mean(axis=0)
    most_important_feature = columns_of_interest[np.argmax(shap_importance)]
    
    # Log the most important feature
    mlflow.log_param("most_important_feature", most_important_feature)
    
    # Predict the values on the combined training and validation data
    y_pred_train_val = best_model.predict(X_train_val)
    
    # Calculate metrics on the combined training and validation data
    mae_train_val = mean_absolute_error(y_train_val, y_pred_train_val)
    mse_train_val = mean_squared_error(y_train_val, y_pred_train_val)
    r2_train_val = r2_score(y_train_val, y_pred_train_val)
    
    # Log training metrics
    mlflow.log_metric("Train_Val_MAE", mae_train_val)
    mlflow.log_metric("Train_Val_MSE", mse_train_val)
    mlflow.log_metric("Train_Val_R2", r2_train_val)
    
    # Log the model with input example
    input_example = X_train_val.iloc[:5]
    mlflow.xgboost.log_model(best_model, "best_model", input_example=input_example)
    
    # Print the most important feature and training metrics
    print(f"The most important feature is {most_important_feature}")
    print(f"Training Metrics: MAE = {mae_train_val:.4f}, MSE = {mse_train_val:.4f}, R² = {r2_train_val:.4f}")
    
    # Plot the SHAP values
    shap.summary_plot(shap_values, X_train_val, feature_names=columns_of_interest, show=False)
    plt.savefig("shap_summary.png", bbox_inches='tight')
    mlflow.log_artifact("shap_summary.png")
    plt.close()
    
    shap.dependence_plot(
        most_important_feature, shap_values.values, X_train_val,
        feature_names=columns_of_interest, show=False
    )
    plt.savefig("shap_dependence.png", bbox_inches='tight')
    mlflow.log_artifact("shap_dependence.png")
    plt.close()
    
    # Save the SHAP values
    shap_values_file = "shap_values.pkl"
    with open(shap_values_file, "wb") as f:
        pickle.dump(shap_values, f)
    mlflow.log_artifact(shap_values_file)
    
    # Log the experimental setup
    mlflow.log_param("experimental_setup", {
        "data_path": path,
        "columns_of_interest": columns_of_interest,
        "test_size": 0.1  # Indicate that 10% of data was used for testing
    })
    
    # Log the preprocessing steps
    mlflow.log_param("preprocessing_steps", {
        "iteration_columns_conversion": "Converted to numeric and replaced zeros with NaN",
        "soh_calculation": "Calculated SoH and created thresholds",
        "data_splitting": "Split data into training+validation and test sets"
    })
    
    # Log the model training and evaluation details
    mlflow.log_param("model_training", {
        "model_type": "XGBoost",
        "evaluation_metrics": ["MAE", "MSE", "R²"],
        "hyperparameter_tuning": "Used Ray Tune with HyperOptSearch and HyperBandScheduler, maximizing R²"
    })
    
    # Track the specific file of the experimental result
    mlflow.log_artifact(path)


In [ ]:
!pip install ray[tune]

import hyperopt
import mlflow
import mlflow.xgboost
import shap
import xgboost as xgb
import numpy as np
import pandas as pd
import pickle
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from ray import tune, train
from ray.tune.schedulers import HyperBandScheduler
from ray.tune.search.hyperopt import HyperOptSearch
from ray.air.integrations.mlflow import MLflowLoggerCallback

# Suppress XGBoost model format warning
warnings.filterwarnings(action='ignore', category=UserWarning, module='xgboost')

# Initialize MLflow experiment
mlflow.set_experiment("New SoH Prediction Experiment")

# Define the search space for hyperparameters, including 'threshold'
search_space = {
    "learning_rate": hyperopt.hp.loguniform("learning_rate", np.log(0.01), np.log(0.1)),
    "max_depth": hyperopt.hp.randint("max_depth", 3, 10),
    "min_child_weight": hyperopt.hp.randint("min_child_weight", 1, 6),
    "subsample": hyperopt.hp.uniform("subsample", 0.5, 1.0),
    "colsample_bytree": hyperopt.hp.uniform("colsample_bytree", 0.5, 1.0),
    "threshold": hyperopt.hp.choice("threshold", [0.98, 0.95, 0.9, 0.85, 0.8])  # Threshold as a hyperparameter
}

# Start an MLflow run
with mlflow.start_run() as parent_run:
    # Log parameters
    path = "Synthetic_data/experiment_results.csv"
    mlflow.log_param("data_path", path)

    # Load data
    df = pd.read_csv(path)
    mlflow.log_param("data_shape", df.shape)

    # Define the columns of interest (features)
    columns_of_interest = [
        'charge_c_rate_modulation', 'protocol_choice_prob', 'charge_soc_modulation',
        'rest_duration_modulation', 'discharge_factor_modulation', 'discharge_soc_modulation'
    ]

    # Log the features used
    mlflow.log_param("columns_of_interest", columns_of_interest)

    # Prepare the data
    data = df.copy()

    # Split the data into training+validation (90%) and test (10%) sets
    train_val_data, test_data = train_test_split(
        data, test_size=0.1, random_state=42
    )

    # Log the sizes of the datasets
    mlflow.log_param("train_val_data_shape", train_val_data.shape)
    mlflow.log_param("test_data_shape", test_data.shape)

    # Define the training function
    def train_model(config, train_val_data, columns_of_interest):
        import xgboost as xgb
        from sklearn.metrics import r2_score
        from sklearn.model_selection import train_test_split
        import pandas as pd
        import numpy as np

        # Extract the threshold from the config
        threshold = config["threshold"]

        # Preprocess data based on the threshold
        # Calculate SoH (State of Health) based on the iteration columns
        iteration_columns = train_val_data.filter(regex='^Iteration').copy()
        iteration_columns = iteration_columns.apply(pd.to_numeric, errors='coerce')
        iteration_columns = iteration_columns.replace(0, np.nan)
        soh = iteration_columns.div(iteration_columns.iloc[:, 0], axis=0)

        # Calculate the target variable 'y' based on the threshold
        soh_threshold = soh.apply(
            lambda row: next((i for i, v in enumerate(row) if v <= threshold), len(row)),
            axis=1
        )
        soh_threshold = soh_threshold.astype(float)

        # Combine features and target into a single DataFrame
        df_with_threshold = train_val_data[columns_of_interest].copy()
        df_with_threshold['target'] = soh_threshold
        df_with_threshold = df_with_threshold.dropna()

        # Split features and target
        X = df_with_threshold[columns_of_interest]
        y = df_with_threshold['target']

        # Split data into training and validation sets (within train_val_data)
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

        # Initialize the XGBoost regressor with hyperparameters from 'config'
        model = xgb.XGBRegressor(
            learning_rate=config["learning_rate"],
            max_depth=int(config["max_depth"]),
            min_child_weight=int(config["min_child_weight"]),
            subsample=config["subsample"],
            colsample_bytree=config["colsample_bytree"],
            objective='reg:squarederror',
            seed=42,
            n_jobs=1  # Limit to 1 CPU per trial to prevent over-subscription
        )

        # Train the model
        model.fit(X_train, y_train)

        # Predict on the validation set
        y_pred = model.predict(X_val)

        # Calculate R² on the validation set
        r2 = r2_score(y_val, y_pred)

        # Report the metric to Ray Tune
        train.report({"r2":r2})

    # Initialize HyperOptSearch
    hyperopt_search = HyperOptSearch(
        space=search_space,
        metric="r2",
        mode="max"
    )

    # Initialize HyperBandScheduler for iterative, race-like optimization
    scheduler = HyperBandScheduler(
        metric="r2",
        mode="max",
        max_t=10,
        reduction_factor=3
    )

    # Set up MLflowLoggerCallback
    mlflow_logger = MLflowLoggerCallback(
        tracking_uri=mlflow.get_tracking_uri(),
        experiment_name=mlflow.get_experiment(mlflow.active_run().info.experiment_id).name,
        save_artifact=True
    )

    # Run hyperparameter tuning
    analysis = tune.run(
        tune.with_parameters(train_model, train_val_data=train_val_data, columns_of_interest=columns_of_interest),
        search_alg=hyperopt_search,
        num_samples=50,  # Number of hyperparameter configurations to try
        scheduler=scheduler,
        resources_per_trial={"cpu": 1},  # Adjust based on your resources
        verbose=1,
        callbacks=[mlflow_logger],
        name="Ray_Tune_Hyperparameter_Search"
    )

    # Get the best hyperparameters
    best_config = analysis.get_best_config(metric="r2", mode="max")

    # Log the best hyperparameters
    mlflow.log_params(best_config)

    # Extract the best threshold
    best_threshold = best_config["threshold"]

    # Recompute the target variable 'y' using the best threshold on the combined train_val_data
    iteration_columns = train_val_data.filter(regex='^Iteration').copy()
    iteration_columns = iteration_columns.apply(pd.to_numeric, errors='coerce')
    iteration_columns = iteration_columns.replace(0, np.nan)
    soh = iteration_columns.div(iteration_columns.iloc[:, 0], axis=0)

    soh_threshold = soh.apply(
        lambda row: next((i for i, v in enumerate(row) if v <= best_threshold), len(row)),
        axis=1
    )
    soh_threshold = soh_threshold.astype(float)

    # Prepare the final dataset with the best threshold
    df_with_threshold = train_val_data[columns_of_interest].copy()
    df_with_threshold['target'] = soh_threshold
    df_with_threshold = df_with_threshold.dropna()

    # Split features and target
    X_train_val = df_with_threshold[columns_of_interest]
    y_train_val = df_with_threshold['target']

    # Retrain the final model on the combined training and validation data
    best_model = xgb.XGBRegressor(
        learning_rate=best_config["learning_rate"],
        max_depth=int(best_config["max_depth"]),
        min_child_weight=int(best_config["min_child_weight"]),
        subsample=best_config["subsample"],
        colsample_bytree=best_config["colsample_bytree"],
        objective='reg:squarederror',
        seed=42
    )

    best_model.fit(X_train_val, y_train_val)

    # Prepare the test set for evaluation
    # Preprocess the test data using the best threshold
    iteration_columns_test = test_data.filter(regex='^Iteration').copy()
    iteration_columns_test = iteration_columns_test.apply(pd.to_numeric, errors='coerce')
    iteration_columns_test = iteration_columns_test.replace(0, np.nan)
    soh_test = iteration_columns_test.div(iteration_columns_test.iloc[:, 0], axis=0)

    soh_threshold_test = soh_test.apply(
        lambda row: next((i for i, v in enumerate(row) if v <= best_threshold), len(row)),
        axis=1
    )
    soh_threshold_test = soh_threshold_test.astype(float)

    # Prepare the test dataset with the best threshold
    df_test_with_threshold = test_data[columns_of_interest].copy()
    df_test_with_threshold['target'] = soh_threshold_test
    df_test_with_threshold = df_test_with_threshold.dropna()

    # Split features and target for the test set
    X_test = df_test_with_threshold[columns_of_interest]
    y_test = df_test_with_threshold['target']

    # Evaluate the model on the test set
    y_pred_test = best_model.predict(X_test)

    # Calculate metrics on the test set
    mae_test = mean_absolute_error(y_test, y_pred_test)
    mse_test = mean_squared_error(y_test, y_pred_test)
    rmse_test = np.sqrt(mse_test)  # Calculate RMSE
    r2_test = r2_score(y_test, y_pred_test)

    # Log test metrics
    mlflow.log_metric("Test_MAE", mae_test)
    mlflow.log_metric("Test_RMSE", rmse_test)
    mlflow.log_metric("Test_R2", r2_test)

    # Print the test metrics
    print(f"Test Metrics: MAE = {mae_test:.4f}, RMSE = {rmse_test:.4f}, R² = {r2_test:.4f}")

    # Calculate SHAP values on the combined training and validation data
    explainer = shap.Explainer(best_model)
    shap_values = explainer(X_train_val)

    # Determine the most important feature
    shap_importance = np.abs(shap_values.values).mean(axis=0)
    most_important_feature = columns_of_interest[np.argmax(shap_importance)]

    # Log the most important feature
    mlflow.log_param("most_important_feature", most_important_feature)

    # Predict the values on the combined training and validation data
    y_pred_train_val = best_model.predict(X_train_val)

    # Calculate metrics on the combined training and validation data
    mae_train_val = mean_absolute_error(y_train_val, y_pred_train_val)
    mse_train_val = mean_squared_error(y_train_val, y_pred_train_val)
    rmse_train_val = np.sqrt(mse_train_val)  # Calculate RMSE
    r2_train_val = r2_score(y_train_val, y_pred_train_val)

    # Log training metrics
    mlflow.log_metric("Train_Val_MAE", mae_train_val)
    mlflow.log_metric("Train_Val_RMSE", rmse_train_val)
    mlflow.log_metric("Train_Val_R2", r2_train_val)

    # Log the model with input example
    input_example = X_train_val.iloc[:5]
    mlflow.xgboost.log_model(best_model, "best_model", input_example=input_example)

    # Print the most important feature and training metrics
    print(f"The most important feature is {most_important_feature}")
    print(f"Training Metrics: MAE = {mae_train_val:.4f}, RMSE = {rmse_train_val:.4f}, R² = {r2_train_val:.4f}")

    # Plot the SHAP values
    shap.summary_plot(shap_values, X_train_val, feature_names=columns_of_interest, show=False)
    plt.savefig("shap_summary.png", bbox_inches='tight')
    mlflow.log_artifact("shap_summary.png")
    plt.close()

    shap.dependence_plot(
        most_important_feature, shap_values.values, X_train_val,
        feature_names=columns_of_interest, show=False
    )
    plt.savefig("shap_dependence.png", bbox_inches='tight')
    mlflow.log_artifact("shap_dependence.png")
    plt.close()

    # Save the SHAP values
    shap_values_file = "shap_values.pkl"
    with open(shap_values_file, "wb") as f:
        pickle.dump(shap_values, f)
    mlflow.log_artifact(shap_values_file)

    # Log the experimental setup
    mlflow.log_param("experimental_setup", {
        "data_path": path,
        "columns_of_interest": columns_of_interest,
        "test_size": 0.1  # Indicate that 10% of data was used for testing
    })

    # Log the preprocessing steps
    mlflow.log_param("preprocessing_steps", {
        "iteration_columns_conversion": "Converted to numeric and replaced zeros with NaN",
        "soh_calculation": "Calculated SoH and created thresholds",
        "data_splitting": "Split data into training+validation and test sets"
    })

    # Log the model training and evaluation details
    mlflow.log_param("model_training", {
        "model_type": "XGBoost",
        "evaluation_metrics": ["MAE", "RMSE", "R²"],
        "hyperparameter_tuning": "Used Ray Tune with HyperOptSearch and HyperBandScheduler, maximizing R²"
    })

    # Track the specific file of the experimental result
    mlflow.log_artifact(path)


In [ ]:
!pip install "ray[tune]"

In [ ]:
import random
import mlflow
import mlflow.xgboost
import xgboost as xgb
import optuna
import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
!pip install optuna
# Set up MLflow experiment
experiment_name = "New SoH Prediction Experiment with Optuna"
mlflow.set_experiment(experiment_name)

# Load data
path = "Synthetic_data/experiment_results.csv"
df = pd.read_csv(path)

# Define the columns of interest (features)
columns_of_interest = [
    'charge_c_rate_modulation', 'protocol_choice_prob', 'charge_soc_modulation',
    'rest_duration_modulation', 'discharge_factor_modulation', 'discharge_soc_modulation', 
]

# Prepare the data
data = df.copy()

# Split the data into training+validation (90%) and test (10%) sets
train_val_data, test_data = train_test_split(
    data, test_size=0.1, random_state=42
)

# Start a parent MLflow run
with mlflow.start_run(run_name="Optuna_Hyperparameter_Tuning") as parent_run:
    parent_run_id = parent_run.info.run_id

    # Define the objective function
    def objective(trial):
        # Start a nested MLflow run for this trial
        with mlflow.start_run(run_name=f"trial_{trial.number}", nested=True):
            # Suggest hyperparameters
            learning_rate = trial.suggest_float('learning_rate', 0.01, 0.1, log=True)
            max_depth = trial.suggest_int('max_depth', 3, 10)
            min_child_weight = trial.suggest_int('min_child_weight', 1, 6)
            subsample = trial.suggest_float('subsample', 0.5, 1.0)
            colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
            threshold = trial.suggest_categorical('threshold', [0.98, 0.95, 0.9, 0.85, 0.8])

            # Log hyperparameters
            mlflow.log_params({
                'learning_rate': learning_rate,
                'max_depth': max_depth,
                'min_child_weight': min_child_weight,
                'subsample': subsample,
                'colsample_bytree': colsample_bytree,
                'threshold': threshold
            })

            # Preprocess data based on the threshold
            iteration_columns = train_val_data.filter(regex='^Iteration').copy()
            iteration_columns = iteration_columns.apply(pd.to_numeric, errors='coerce')
            iteration_columns = iteration_columns.replace(0, np.nan)
            soh = iteration_columns.div(iteration_columns.iloc[:, 0], axis=0)

            # Calculate the target variable 'y' based on the threshold
            soh_threshold = soh.apply(
                lambda row: next((i for i, v in enumerate(row) if v <= threshold), len(row)),
                axis=1
            ).astype(float)

            # Prepare the dataset
            df_with_threshold = train_val_data[columns_of_interest].copy()
            df_with_threshold['target'] = soh_threshold
            df_with_threshold = df_with_threshold.dropna()

            X = df_with_threshold[columns_of_interest]
            y = df_with_threshold['target']

            # Split data into training and validation sets
            X_train, X_val, y_train, y_val = train_test_split(
                X, y, test_size=0.2, random_state=42
            )

            # Initialize the XGBoost regressor
            model = xgb.XGBRegressor(
                learning_rate=learning_rate,
                max_depth=max_depth,
                min_child_weight=min_child_weight,
                subsample=subsample,
                colsample_bytree=colsample_bytree,
                objective='reg:squarederror',
                seed=42
            )

            # Train the model
            model.fit(X_train, y_train)

            # Predict on the validation set
            y_pred = model.predict(X_val)

            # Calculate metrics on the validation set
            r2 = r2_score(y_val, y_pred)
            mae_val = mean_absolute_error(y_val, y_pred)
            rmse_val = np.sqrt(mean_squared_error(y_val, y_pred))

            # Log metrics
            mlflow.log_metrics({
                'validation_R2': r2,
                'validation_MAE': mae_val,
                'validation_RMSE': rmse_val
            })

            # Return the validation R² to be maximized
            return rmse_val

    # Remove MLflowCallback since we're handling logging manually
    # Create an Optuna study
    study = optuna.create_study(
        direction='minimize',
        study_name='XGBoost_SoH_Optimization',
        sampler=optuna.samplers.TPESampler(seed=42)
    )

    # Optimize the objective function
    study.optimize(objective, n_trials=50)

    # Get the best hyperparameters
    best_params = study.best_params

    # Extract the best threshold
    best_threshold = best_params.pop('threshold')

    # Recompute the target variable 'y' using the best threshold on the combined train_val_data
    iteration_columns = train_val_data.filter(regex='^Iteration').copy()
    iteration_columns = iteration_columns.apply(pd.to_numeric, errors='coerce')
    iteration_columns = iteration_columns.replace(0, np.nan)
    soh = iteration_columns.div(iteration_columns.iloc[:, 0], axis=0)

    soh_threshold = soh.apply(
        lambda row: next((i for i, v in enumerate(row) if v <= best_threshold), len(row)),
        axis=1
    ).astype(float)

    # Prepare the final dataset with the best threshold
    df_with_threshold = train_val_data[columns_of_interest].copy()
    df_with_threshold['target'] = soh_threshold
    df_with_threshold = df_with_threshold.dropna()

    X_train_val = df_with_threshold[columns_of_interest]
    y_train_val = df_with_threshold['target']

    # Initialize the final model
    final_model = xgb.XGBRegressor(
        objective='reg:squarederror',
        seed=42,
        **best_params  # Unpack the best hyperparameters
    )

    # Train the final model
    final_model.fit(X_train_val, y_train_val)

    # Prepare the test set
    iteration_columns_test = test_data.filter(regex='^Iteration').copy()
    iteration_columns_test = iteration_columns_test.apply(pd.to_numeric, errors='coerce')
    iteration_columns_test = iteration_columns_test.replace(0, np.nan)
    soh_test = iteration_columns_test.div(iteration_columns_test.iloc[:, 0], axis=0)

    soh_threshold_test = soh_test.apply(
        lambda row: next((i for i, v in enumerate(row) if v <= best_threshold), len(row)),
        axis=1
    ).astype(float)

    df_test_with_threshold = test_data[columns_of_interest].copy()
    df_test_with_threshold['target'] = soh_threshold_test
    df_test_with_threshold = df_test_with_threshold.dropna()

    X_test = df_test_with_threshold[columns_of_interest]
    y_test = df_test_with_threshold['target']

    # Evaluate the model on the test set
    y_pred_test = final_model.predict(X_test)

    # Calculate metrics on the test set
    mae_test = mean_absolute_error(y_test, y_pred_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
    r2_test = r2_score(y_test, y_pred_test)

    # Start a nested MLflow run for the final model
    with mlflow.start_run(run_name="Final_Model", nested=True):
        # Log best hyperparameters
        mlflow.log_params(best_params)
        mlflow.log_param("threshold", best_threshold)

        # Log test metrics
        mlflow.log_metrics({
            "Test_MAE": mae_test,
            "Test_RMSE": rmse_test,
            "Test_R2": r2_test
        })

        # Calculate SHAP values
        explainer = shap.Explainer(final_model)
        shap_values = explainer(X_train_val)

        # Determine the most important feature
        shap_importance = np.abs(shap_values.values).mean(axis=0)
        most_important_feature = columns_of_interest[np.argmax(shap_importance)]
        mlflow.log_param("most_important_feature", most_important_feature)

        # Log the model
        mlflow.xgboost.log_model(final_model, "final_model")

        # Plot SHAP summary
        shap.summary_plot(shap_values, X_train_val, feature_names=columns_of_interest, show=False)
        plt.savefig("shap_summary.png", bbox_inches='tight')
        mlflow.log_artifact("shap_summary.png")
        plt.close()

        # Plot SHAP dependence for the most important feature
        shap.dependence_plot(
            most_important_feature, shap_values.values, X_train_val,
            feature_names=columns_of_interest, show=False
        )
        plt.savefig("shap_dependence.png", bbox_inches='tight')
        mlflow.log_artifact("shap_dependence.png")
        plt.close()

        # Save SHAP values
        shap_values_file = "shap_values.pkl"
        with open(shap_values_file, "wb") as f:
            pickle.dump(shap_values, f)
        mlflow.log_artifact(shap_values_file)

        # Log data information
        mlflow.log_param("data_path", path)
        mlflow.log_param("data_shape", df.shape)
        mlflow.log_param("columns_of_interest", columns_of_interest)
        mlflow.log_param("train_val_data_shape", train_val_data.shape)
        mlflow.log_param("test_data_shape", test_data.shape)


In [ ]:
import random
import mlflow
import mlflow.xgboost
import xgboost as xgb
import optuna
import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Set up MLflow experiment
experiment_name = "New SoH Prediction Experiment with Optuna"
mlflow.set_experiment(experiment_name)

# Load data
path = "Synthetic_data/experiment_results.csv"
df = pd.read_csv(path)

#
# Define the columns of interest (features)
columns_of_interest = [
    'charge_c_rate_modulation', 'protocol_choice_prob', 'charge_soc_modulation',
    'rest_duration_modulation', 'discharge_factor_modulation', 'discharge_soc_modulation', 'discharge_column_prob'
]

# Prepare the data
data = df.copy()

# Split the data into training+validation (90%) and test (10%) sets
train_val_data, test_data = train_test_split(
    data, test_size=0.1, random_state=42
)

# Start a parent MLflow run
with mlflow.start_run(run_name="Optuna_Hyperparameter_Tuning") as parent_run:
    parent_run_id = parent_run.info.run_id

    # Define the objective function
    def objective(trial):
        # Start a nested MLflow run for this trial
        with mlflow.start_run(run_name=f"trial_{trial.number}", nested=True):
            # Suggest hyperparameters
            learning_rate = trial.suggest_float('learning_rate', 0.01, 0.1, log=True)
            max_depth = trial.suggest_int('max_depth', 3, 10)
            min_child_weight = trial.suggest_int('min_child_weight', 1, 6)
            subsample = trial.suggest_float('subsample', 0.5, 1.0)
            colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
            threshold = trial.suggest_categorical('threshold', [0.98, 0.95, 0.9, 0.85, 0.8])

            # Log hyperparameters
            mlflow.log_params({
                'learning_rate': learning_rate,
                'max_depth': max_depth,
                'min_child_weight': min_child_weight,
                'subsample': subsample,
                'colsample_bytree': colsample_bytree,
                'threshold': threshold
            })

            # Preprocess data based on the threshold
            iteration_columns = train_val_data.filter(regex='^Iteration').copy()
            iteration_columns = iteration_columns.apply(pd.to_numeric, errors='coerce')
            iteration_columns = iteration_columns.replace(0, np.nan)
            soh = iteration_columns.div(iteration_columns.iloc[:, 0], axis=0)

            # Calculate the target variable 'y' based on the threshold
            soh_threshold = soh.apply(
                lambda row: next((i for i, v in enumerate(row) if v <= threshold), len(row)),
                axis=1
            ).astype(float)

            # Prepare the dataset
            df_with_threshold = train_val_data[columns_of_interest].copy()
            df_with_threshold['target'] = soh_threshold
            df_with_threshold = df_with_threshold.dropna()
             # **Discretize and convert to categorical dtype**
            # Discretize 'protocol_choice_prob' into 20 categories
            df_with_threshold['protocol_choice_prob'] = pd.cut(
                df_with_threshold['protocol_choice_prob'],
                bins=10,
                labels=False,
                include_lowest=True
            ).astype('category')

            # Discretize 'discharge_column_prob' into 100 categories
            df_with_threshold['discharge_column_prob'] = pd.cut(
                df_with_threshold['discharge_column_prob'],
                bins=40,
                labels=False,
                include_lowest=True
            ).astype('category')

            X = df_with_threshold[columns_of_interest]
            y = df_with_threshold['target']

            # Split data into training and validation sets
            X_train, X_val, y_train, y_val = train_test_split(
                X, y, test_size=0.2, random_state=42
            )

            # Initialize the XGBoost regressor
            model = xgb.XGBRegressor(
                learning_rate=learning_rate,
                max_depth=max_depth,
                min_child_weight=min_child_weight,
                subsample=subsample,
                colsample_bytree=colsample_bytree,
                objective='reg:squarederror',
                seed=42,
                enable_categorical=True
            )

            # Train the model
            model.fit(X_train, y_train)

            # Predict on the validation set
            y_pred = model.predict(X_val)

            # Calculate metrics on the validation set
            r2 = r2_score(y_val, y_pred)
            mae_val = mean_absolute_error(y_val, y_pred)
            rmse_val = np.sqrt(mean_squared_error(y_val, y_pred))

            # Log metrics
            mlflow.log_metrics({
                'validation_R2': r2,
                'validation_MAE': mae_val,
                'validation_RMSE': rmse_val
            })

            # Return the validation R² to be maximized
            return rmse_val

    # Remove MLflowCallback since we're handling logging manually
    # Create an Optuna study
    study = optuna.create_study(
        direction='minimize',
        study_name='XGBoost_SoH_Optimization',
        sampler=optuna.samplers.TPESampler(seed=42)
    )

    # Optimize the objective function
    study.optimize(objective, n_trials=300)

    # Get the best hyperparameters
    best_params = study.best_params

    # Extract the best threshold
    best_threshold = best_params.pop('threshold')

    # Recompute the target variable 'y' using the best threshold on the combined train_val_data
    iteration_columns = train_val_data.filter(regex='^Iteration').copy()
    iteration_columns = iteration_columns.apply(pd.to_numeric, errors='coerce')
    iteration_columns = iteration_columns.replace(0, np.nan)
    soh = iteration_columns.div(iteration_columns.iloc[:, 0], axis=0)

    soh_threshold = soh.apply(
        lambda row: next((i for i, v in enumerate(row) if v <= best_threshold), len(row)),
        axis=1
    ).astype(float)

    # Prepare the final dataset with the best threshold
    df_with_threshold = train_val_data[columns_of_interest].copy()
    df_with_threshold['target'] = soh_threshold
    df_with_threshold = df_with_threshold.dropna()
    # Discretize and convert to categorical dtype
    df_with_threshold['protocol_choice_prob'] = pd.cut(
        df_with_threshold['protocol_choice_prob'],
        bins=10,
        labels=False,
        include_lowest=True
    ).astype('category')

    df_with_threshold['discharge_column_prob'] = pd.cut(
        df_with_threshold['discharge_column_prob'],
        bins=40,
        labels=False,
        include_lowest=True
    ).astype('category')
    X_train_val = df_with_threshold[columns_of_interest]
    y_train_val = df_with_threshold['target']

    final_model = xgb.XGBRegressor(
        objective='reg:squarederror',
        seed=42,
        enable_categorical=True,
        **best_params  # Unpack the best hyperparameters
    )
    

    # Train the final model
    final_model.fit(X_train_val, y_train_val)

    # Prepare the test set
    iteration_columns_test = test_data.filter(regex='^Iteration').copy()
    iteration_columns_test = iteration_columns_test.apply(pd.to_numeric, errors='coerce')
    iteration_columns_test = iteration_columns_test.replace(0, np.nan)
    soh_test = iteration_columns_test.div(iteration_columns_test.iloc[:, 0], axis=0)

    soh_threshold_test = soh_test.apply(
        lambda row: next((i for i, v in enumerate(row) if v <= best_threshold), len(row)),
        axis=1
    ).astype(float)

    df_test_with_threshold = test_data[columns_of_interest].copy()
    df_test_with_threshold['target'] = soh_threshold_test
    df_test_with_threshold = df_test_with_threshold.dropna()

    X_test = df_test_with_threshold[columns_of_interest]
    y_test = df_test_with_threshold['target']

    # Evaluate the model on the test set
    y_pred_test = final_model.predict(X_test)

    # Calculate metrics on the test set
    mae_test = mean_absolute_error(y_test, y_pred_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
    r2_test = r2_score(y_test, y_pred_test)

    # Start a nested MLflow run for the final model
    with mlflow.start_run(run_name="Final_Model", nested=True):
        # Log best hyperparameters
        mlflow.log_params(best_params)
        mlflow.log_param("threshold", best_threshold)

        # Log test metrics
        mlflow.log_metrics({
            "Test_MAE": mae_test,
            "Test_RMSE": rmse_test,
            "Test_R2": r2_test
        })

        # Calculate SHAP values
        explainer = shap.Explainer(final_model)
        shap_values = explainer(X_train_val)

        # Determine the most important feature
        shap_importance = np.abs(shap_values.values).mean(axis=0)
        most_important_feature = columns_of_interest[np.argmax(shap_importance)]
        mlflow.log_param("most_important_feature", most_important_feature)

        # Log the model
        mlflow.xgboost.log_model(final_model, "final_model")

        # Plot SHAP summary
        shap.summary_plot(shap_values, X_train_val, feature_names=columns_of_interest, show=False)
        plt.savefig("shap_summary.png", bbox_inches='tight')
        mlflow.log_artifact("shap_summary.png")
        plt.close()

        # Plot SHAP dependence for the most important feature
        shap.dependence_plot(
            most_important_feature, shap_values.values, X_train_val,
            feature_names=columns_of_interest, show=False
        )
        plt.savefig("shap_dependence.png", bbox_inches='tight')
        mlflow.log_artifact("shap_dependence.png")
        plt.close()

        # Save SHAP values
        shap_values_file = "shap_values.pkl"
        with open(shap_values_file, "wb") as f:
            pickle.dump(shap_values, f)
        mlflow.log_artifact(shap_values_file)

        # Log data information
        mlflow.log_param("data_path", path)
        mlflow.log_param("data_shape", df.shape)
        mlflow.log_param("columns_of_interest", columns_of_interest)
        mlflow.log_param("train_val_data_shape", train_val_data.shape)
        mlflow.log_param("test_data_shape", test_data.shape)

